In [1]:
import pandas as pd

In [3]:
data_df = pd.read_csv("top.csv",index_col=0, parse_dates=True)

In [26]:
def low(day):
    return "minute_low" if day == 2 else "low" + str(day)
def yz(day):
    return (data_df[low(day)] == data_df["high_limit" + str(day)])
def notyz(day):
    return (data_df[low(day)] < data_df["high_limit" + str(day)])
def top(day):
    return (data_df["close" + str(day)] == data_df["high_limit" + str(day)])
def nottop(day):
    return (data_df["close" + str(day)] < data_df["high_limit" + str(day)])
def opentop(day):
    return (data_df["open" + str(day)] == data_df["high_limit" + str(day)])
#前天/昨天/今天开盘涨停
opentop0 = opentop(0)
opentop1 = opentop(1)
opentop2 = opentop(2)
# 前天/昨天/今天是/不是一字板
yz0 = yz(0)
yz1 = yz(1)
notyz0 = notyz(0)
notyz1 = notyz(1)
notyz2 = notyz(2)
# 前天/昨天是/否涨停
top0 = top(0)
top1 = top(1)
nottop0 = nottop(0)
nottop1 = nottop(1)
def jump(day):
    return (data_df[low(day)] > data_df["high" + str(day - 1)]) 
def foot(day,degree = 1.0):
    return (data_df[low(day)] >= data_df["open" + str(day)] * degree)
# 昨天/今天 跳高
jump1 = jump(1)
jump2 = jump(2)
# 前天/昨天/今天 光脚
foot0 = foot(0,0.995)
foot1 = foot(1,0.995)
foot2 = foot(2,0.995)
def speedup(day):
    return (data_df["high" + str(day)] - data_df[low(day)] < data_df["high" + str(day - 1)] - data_df[low(day - 1)])

#昨天/今天 加速
speedup1 = speedup(1)
speedup2 = speedup(2)


def recent(day):
    return (data_df["recent"] == day)
recent5 = recent(5)
recent4 = recent(4)
recent3 = recent(3)


minute = (data_df["minute"] < "11:00:00")
volume = (data_df["minute_volume"] < data_df["volume1"] * 0.5)

In [82]:
def result(valid_data_df):
    result_df = pd.DataFrame()
    result_df["mean"] = valid_data_df.groupby(["recent"]).change.mean()
    result_df["count"] = valid_data_df.groupby(["recent"]).change.count()
    result_df = result_df[result_df["count"] > 5]
    print result_df
    win_df = valid_data_df[valid_data_df["change"] > 1.0]
    win_ratio = float(len(win_df))/len(valid_data_df)
    print win_ratio,len(valid_data_df),valid_data_df.change.mean()
#     valid_data_df = valid_data_df.sort_index()
#     value_se = valid_data_df.change.cumprod()
#     print value_se.plot(figsize=(7,4))

In [129]:
filter1 = opentop0 & notyz0 & ~opentop1 & ~foot1 & foot2 & speedup2 & volume & notyz2 & recent5
filter2 = opentop0 & notyz0 & ~opentop1 & ~foot1 & speedup2 & volume & notyz2 & recent5

In [96]:
len(valid_df[filter1])

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


IndexingError: Unalignable boolean Series key provided

In [128]:
valid_df = data_df[filter1]
valid_df = valid_df.sort_values("minute")
valid_df = valid_df.groupby("date").first()
result(valid_df)

            mean  count
recent                 
5.0     1.132001     14
0.857142857143 14 1.13200063708


In [126]:
valid_df = data_df[filter2]
valid_df = valid_df.sort_values("minute")
valid_df = valid_df.groupby("date").first()
result(valid_df)

            mean  count
recent                 
5.0     1.099402     24
0.833333333333 24 1.09940167683


In [13]:
valid_df = valid_df[valid_df["recent"] == 5.0]
valid_df.sort()

,change,stock,close0,high0,high_limit0,low0,open0,volume0,day0,close1,...,high3,high_limit3,low3,open3,volume3,day3,recent,minute_low,minute_volume,minute
date,,,,,,,,,,,,,,,,,,,,,
2007-03-16 00:00:00,0.924078,600093.XSHG,4.18,4.18,4.18,3.90,4.18,22395348.0,44.0,4.39,...,4.40,4.630000e+00,4.19,4.26,14883006.0,47.0,5.0,4.60,0.0,09:30:00
2007-04-17 00:00:00,2.421053,000732.XSHE,3.10,3.10,3.10,3.00,3.10,16760311.0,66.0,3.26,...,12.42,1.073742e+09,8.28,8.28,67568960.0,69.0,5.0,3.36,0.0,09:30:00
2007-05-18 00:00:00,1.209699,600848.XSHG,12.10,12.10,12.10,11.38,12.10,6241057.0,81.0,13.31,...,16.10,1.610000e+01,16.10,16.10,1142958.0,84.0,5.0,14.31,748100.0,09:32:00
2007-05-28 00:00:00,1.050199,600599.XSHG,11.38,11.38,11.38,10.91,11.38,13176679.0,81.0,11.95,...,13.18,1.318000e+01,12.00,13.18,11562673.0,84.0,5.0,12.48,170306.0,09:31:00
2007-08-13 00:00:00,1.027855,000007.XSHE,6.51,6.51,6.51,6.45,6.51,2489300.0,139.0,6.84,...,7.54,7.540000e+00,7.25,7.38,3185652.0,142.0,5.0,7.17,0.0,09:30:00
2007-08-15 00:00:00,0.899514,600714.XSHG,11.19,11.19,11.19,10.81,11.19,1879408.0,143.0,11.75,...,11.87,1.187000e+01,10.87,11.10,3307820.0,146.0,5.0,12.30,0.0,09:30:00
2008-04-11 00:00:00,1.049656,000605.XSHE,11.88,11.88,11.88,11.75,11.88,2187882.0,87.0,12.47,...,13.74,1.374000e+01,13.20,13.74,3331797.0,90.0,5.0,12.86,192600.0,09:34:00
2008-04-30 00:00:00,1.049307,600338.XSHG,5.89,5.89,5.89,5.65,5.89,795472.0,308.0,6.18,...,6.81,6.810000e+00,6.70,6.81,1620396.0,311.0,5.0,6.38,0.0,10:30:00
2008-05-06 00:00:00,1.042403,600076.XSHG,5.13,5.13,5.13,5.01,5.13,1479603.0,311.0,5.39,...,5.94,5.940000e+00,5.81,5.90,1153644.0,314.0,5.0,5.48,996260.0,09:42:00
